In [1]:
# import pandas for table visualizations
import pandas as pd

In [2]:
mort_exp = pd.read_csv('data/expmortjoin.csv')
mort_exp.head()

,country,code,region,income,year,che_gdp(%),hk_gdl(%),che_pc_usd,che,gdp_pc_usd,che_usd,Year,Number,Percent_of_Total,Age_Stnd_Per_100k,Death_rate_per_100k
0,Argentina,ARG,AMR,Upper-middle,2011,9.4,0.0,"1,203","205,225","12,769","49,931",2011,14572,4.600982,28.986910,35.265788
1,Argentina,ARG,AMR,Upper-middle,2012,9.8,0.0,"1,362","259,235","13,859","57,139",2012,13986,4.409984,27.442447,33.495233
2,Argentina,ARG,AMR,Upper-middle,2013,9.8,0.0,"1,415","327,497","14,469","59,988",2013,14654,4.532254,28.102552,34.728386
3,Argentina,ARG,AMR,Upper-middle,2014,9.7,0.0,"1,281","442,857","13,241","54,841",2014,14816,4.600742,27.769779,34.748745
4,Argentina,ARG,AMR,Upper-middle,2017,10.2,0.0,"1,495","1,090,728","14,610","65,854",2017,14652,4.288123,25.605289,33.347642


In [3]:
# drop columns not being used in testing and the target variables
df_model = mort_exp.drop(columns=['country','code','region','year','Percent_of_Total',
                                  'Age_Stnd_Per_100k','Death_rate_per_100k', 'Year','Number'])
df_model.head()

,income,che_gdp(%),hk_gdl(%),che_pc_usd,che,gdp_pc_usd,che_usd
0,Upper-middle,9.4,0.0,"1,203","205,225","12,769","49,931"
1,Upper-middle,9.8,0.0,"1,362","259,235","13,859","57,139"
2,Upper-middle,9.8,0.0,"1,415","327,497","14,469","59,988"
3,Upper-middle,9.7,0.0,"1,281","442,857","13,241","54,841"
4,Upper-middle,10.2,0.0,"1,495","1,090,728","14,610","65,854"


In [4]:
# get dummy variables for income column
income_dummies = pd.get_dummies(df_model['income'], dtype=int)
income_dummies.head()

,High,Lower-middle,Upper-middle
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1


In [5]:
# combine original df with dummy df and drop original income column

df_dummies = df_model.copy()

df_dummies = pd.concat([df_dummies,income_dummies], axis=1).drop(columns='income')
df_dummies.head()

,che_gdp(%),hk_gdl(%),che_pc_usd,che,gdp_pc_usd,che_usd,High,Lower-middle,Upper-middle
0,9.4,0.0,"1,203","205,225","12,769","49,931",0,0,1
1,9.8,0.0,"1,362","259,235","13,859","57,139",0,0,1
2,9.8,0.0,"1,415","327,497","14,469","59,988",0,0,1
3,9.7,0.0,"1,281","442,857","13,241","54,841",0,0,1
4,10.2,0.0,"1,495","1,090,728","14,610","65,854",0,0,1


In [6]:
# preview data types to make sure everything is as it should be
df_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356 entries, 0 to 355
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   che_gdp(%)    356 non-null    float64
 1   hk_gdl(%)     356 non-null    float64
 2   che_pc_usd    356 non-null    object 
 3   che           356 non-null    object 
 4   gdp_pc_usd    356 non-null    object 
 5   che_usd       356 non-null    object 
 6   High          356 non-null    int64  
 7   Lower-middle  356 non-null    int64  
 8   Upper-middle  356 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 25.2+ KB


In [7]:
# convert object columns to int
df_dummies.replace(',','',regex=True, inplace=True)

df_dummies[['che_pc_usd','che',
            'gdp_pc_usd','che_usd']]= df_dummies[['che_pc_usd','che',
                                                  'gdp_pc_usd','che_usd']].apply(pd.to_numeric)

df_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356 entries, 0 to 355
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   che_gdp(%)    356 non-null    float64
 1   hk_gdl(%)     356 non-null    float64
 2   che_pc_usd    356 non-null    float64
 3   che           356 non-null    int64  
 4   gdp_pc_usd    356 non-null    int64  
 5   che_usd       356 non-null    int64  
 6   High          356 non-null    int64  
 7   Lower-middle  356 non-null    int64  
 8   Upper-middle  356 non-null    int64  
dtypes: float64(3), int64(6)
memory usage: 25.2 KB


In [8]:
# import scikit learn libraries for model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression


In [9]:
# define y variable from original dataframe starting with the percentage of total deaths
total_y = mort_exp['Percent_of_Total']


In [10]:
# use the average and median of this column and use that to catagorize the data and make a new column to predict

print(total_y.mean())
print(total_y.median())


# assign these to variables
total_y_mean = total_y.mean()
total_y_median = total_y.median()

# create empty lists
perc_of_total_avg = []
perc_of_total_med = []

# run a for loop that goes through the total_y array and assigns a 0 (below avg) or 1 (above avg)
for x in total_y:
    if x > total_y_mean:
        perc_of_total_avg.append(1)
    else:
        perc_of_total_avg.append(0)
        
    if x > total_y_median:
        perc_of_total_med.append(1)
        
    else:
        perc_of_total_med.append(0)
        


3.7364148851123598
3.0605352200000002


In [11]:
# split the data into features and target data starting with avg

X = df_dummies
y = perc_of_total_avg


# Split the dataset using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [12]:
# Instantiate a StandardScaler instance
scaler = StandardScaler()

# Fit the training data to the standard scaler
X_scaler = scaler.fit(X_train)

# Transform the training data using the scaler
X_train_scaled = X_scaler.transform(X_train)

# Transform the testing data using the scaler
X_test_scaled = X_scaler.transform(X_test)

### Logistic Regression with Original Data

In [38]:
# Declare a logistic regression model.
# Apply a random_state of 9 to the model
logistic_regression_model = LogisticRegression(solver="lbfgs", random_state=9)

# Fit and save the logistic regression model using the training data
logistic_regression_model.fit(X_train, y_train)

/Users/bbjf/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=9)

In [14]:
#Generate training predictions
training_predictions = lr_model.predict(X_train)

#Generate testing predictions
testing_predictions = logistic_regression_model.predict(X_test)

In [39]:
testing_predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

In [15]:
# Create and save the training classification report
training_report = classification_report(y_train, training_predictions)

# Print the training classification report
print(training_report)

              precision    recall  f1-score   support

           0       0.62      0.98      0.76       160
           1       0.77      0.09      0.17       107

    accuracy                           0.63       267
   macro avg       0.69      0.54      0.46       267
weighted avg       0.68      0.63      0.52       267



TRAINING data classification report, gives us only 67% accuracy

In [16]:
# Create and save the testing classification report
testing_report = classification_report(y_test, testing_predictions)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

           0       0.68      0.97      0.80        59
           1       0.60      0.10      0.17        30

    accuracy                           0.67        89
   macro avg       0.64      0.53      0.48        89
weighted avg       0.65      0.67      0.59        89



### KNeighborsClassifier model

In [17]:
# Import the KNeighborsClassifier module from sklearn
from sklearn.neighbors import KNeighborsClassifier

# Instantiate the KNeighborsClassifier model with n_neighbors = 3 
knn = KNeighborsClassifier(n_neighbors=3)

In [18]:
# Train the model using the training data
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=3)

In [19]:
# Create predictions using the testing data
y_pred = knn.predict(X_test_scaled)

In [20]:
# Print the classification report comparing the testing data to the model predictions
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.86      0.91        59
           1       0.78      0.93      0.85        30

    accuracy                           0.89        89
   macro avg       0.87      0.90      0.88        89
weighted avg       0.90      0.89      0.89        89



### Rinse and repeat for Age_Stnd_Per_100k

In [21]:
# define y variable from original dataframe
age_stnd_y = mort_exp['Age_Stnd_Per_100k']

In [22]:

# use the average and median of this column and use that to catagorize the data and make a new column to predict

print(age_stnd_y.mean())
print(age_stnd_y.median())


# assign these to variables
age_stnd_y_mean = age_stnd_y.mean()
age_stnd_y_median = age_stnd_y.median()

# create empty lists
age_stnd_avg = []
age_stnd_med = []

# run a for loop that goes through the age_stnd_y array and assigns a 0 (below avg) or 1 (above avg)
for x in age_stnd_y:
    if x > age_stnd_y_mean:
        age_stnd_avg.append(1)
    else:
        age_stnd_avg.append(0)
        
    if x > age_stnd_y_mean:
        age_stnd_med.append(1)
        
    else:
        age_stnd_med.append(0)

21.038678909410113
16.511751365000002


The median and mean are pretty far apart. I'm sticking with the mean for now , but will consult with the group if the median is more appropriate.

In [23]:
# split the data into features and target data starting with avg

X = df_dummies
y = age_stnd_avg


# Split the dataset using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [24]:
# Instantiate a StandardScaler instance
scaler = StandardScaler()

# Fit the training data to the standard scaler
X_scaler = scaler.fit(X_train)

# Transform the training data using the scaler
X_train_scaled = X_scaler.transform(X_train)

# Transform the testing data using the scaler
X_test_scaled = X_scaler.transform(X_test)

In [25]:
# Instantiate the KNeighborsClassifier model with n_neighbors = 3 
knn = KNeighborsClassifier(n_neighbors=3)

In [26]:
# Train the model using the training data
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=3)

In [27]:
# Create predictions using the testing data
y_pred = knn.predict(X_test_scaled)

In [28]:
# Print the classification report comparing the testing data to the model predictions
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.90      0.93        59
           1       0.82      0.93      0.88        30

    accuracy                           0.91        89
   macro avg       0.89      0.92      0.90        89
weighted avg       0.92      0.91      0.91        89



### Last time for the Death_rate_per_100k

In [29]:
# define y variable from original dataframe
death_y = mort_exp['Age_Stnd_Per_100k']

In [30]:
# use the average and median of this column and use that to catagorize the data and make a new column to predict

print(death_y.mean())
print(death_y.median())


# assign these to variables
death_y_mean = death_y.mean()
death_y_median = death_y.median()

# create empty lists
death_avg = []
death_med = []

# run a for loop that goes through the age_stnd_y array and assigns a 0 (below avg) or 1 (above avg)
for x in death_y:
    if x > death_y_mean:
        death_avg.append(1)
    else:
        death_avg.append(0)
        
    if x > death_y_median:
        death_med.append(1)
        
    else:
        death_med.append(0)
        
        
        

21.038678909410113
16.511751365000002


Similar results to the age standarized

In [31]:
# split the data into features and target data starting with avg

X = df_dummies
y = death_avg


# Split the dataset using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [32]:
# Instantiate a StandardScaler instance
scaler = StandardScaler()

# Fit the training data to the standard scaler
X_scaler = scaler.fit(X_train)

# Transform the training data using the scaler
X_train_scaled = X_scaler.transform(X_train)

# Transform the testing data using the scaler
X_test_scaled = X_scaler.transform(X_test)

In [33]:
# Instantiate the KNeighborsClassifier model with n_neighbors = 3 
knn = KNeighborsClassifier(n_neighbors=3)

In [34]:
# Train the model using the training data
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=3)

In [35]:
# Create predictions using the testing data
y_pred = knn.predict(X_test_scaled)

In [36]:
# Print the classification report comparing the testing data to the model predictions
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.90      0.93        59
           1       0.82      0.93      0.88        30

    accuracy                           0.91        89
   macro avg       0.89      0.92      0.90        89
weighted avg       0.92      0.91      0.91        89



the age standarized and the death rate per columns have almost identical results